<a href="https://colab.research.google.com/github/liuxy10/CS188MiniContest/blob/master/BLANK_CS188_WHW4_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Written HW 4: Code Portion**

# There are two things you need to do before anything else:

1. File -> Save a Copy in Drive
2. In your own copy of the colab notebook, make sure you use a **GPU**! Go to Runtime -> Change runtime type -> set Hardware accelator to GPU. 

<!--<img src="https://upload.wikimedia.org/wikipedia/commons/b/bb/Defense.gov_photo_essay_100124-D-0000G-001.jpg" alt="A U.S. Navy sailor carries a Haitian boy off a helicopter at Terminal Varreux, Haiti, Jan. 23, 2010, as the boy's mother follows behind them. The child received treatment aboard one of the U.S. Navy ships serving as a hospital in Port-au-Prince harbor, Haiti and he was later discharged. U.S. military personnel are providing aid and support to earthquake victims in Haiti." width="400"/>-->


### Colaboratory

You are currently using a Colaboratory notebook, which runs Jupyter notebooks. There are text cells and code cells, and you'll be writing mostly in code cells, though feel free to double click and edit text cells to write answers to the questions we've sprinkled throughout (marked **Q**). 

A key idea is to make sure you run cells in order, and double check that you've run previous cells before running a current cell. When you declare a variable or function in a cell, you need to run it so the notebook "remembers" it for use in future cells. 

To run a cell, press Shift->Enter or click on the play button in the top left corner of the cell. 

Feel free to go to Tools -> Preferences -> Miscellaneous to adjust some fun settings. 

# Q1: Probabilistic Language Modeling: Coding Portion


These are the functions you need to fill in in the N-gram class. Please only add code between the """ START YOUR CODE HERE """ and """ END YOUR CODE HERE """ comments and don't change anything else or else Staff will have a really hard time helping you debug in OH:
1. count_words
2. calc_word_probs
3. probs_to_neg_log_probs
4. filter_adj_counter
5. p_naive
6. calc_neg_log_prob_of_sentence
7. calc_prob_of_sentence

In [0]:
import nltk
from nltk.tokenize import RegexpTokenizer
import collections
import string
import re
import numpy as np
import random
import matplotlib.pyplot as plt
import time
import itertools

In [0]:
class N_gram:
    def __init__(self, corpora_path_list, N, regexp_tokenizer=True, add_periods=False, case_sensitive=False):
        self.N = N
        self.corpora_path_list = corpora_path_list
        self.text_list = None
        self.adj_counters = None
        self.phrases_sets = None
        self.V = None
        self.regexp_tokenizer = regexp_tokenizer
        self.add_periods = add_periods
        self.case_sensitive = case_sensitive

    def train(self):
        print("Preprocessing training corpora...")
        if self.text_list is None:
            self.text_list = self.preprocess_corpus(self.corpora_path_list)
            print("Training set size:", len(self.text_list), "words")
        if self.case_sensitive:
            print("Recording all variations of capitalization...")
            self.capsVariations = self.recordCapsVariations()
        print("Constructing Phrase/Adjacency counters...")
        self.adj_counters = self.calc_adj_counters(self.N)
        print("Constructing List of Phrase Sets...")
        self.phrases_sets = [set(self.list_phrases(i)) for i in range(self.N + 1)]
        self.V = len(set(self.text_list)) # size of vocab (# unique words in corpora)
        print("Done Training.")

    def preprocess_corpus(self, text_file_path_list):
        """
        Input: String of text_file_path.
        Returns: List format of text, as divided by regex delimiters.
        Either tokenizes punctuation or not, depending on RegexpTokenizer parameter;
        Lowercases all words.
        
        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> c.preprocess_corpus(["carter_concession.txt"])
        ["i", "promised", "you", "four", "years", "ago", ...]
        """
        assert isinstance(text_file_path_list, list), "text_file_path_list not a list."
        def txt_to_str(text_file_path_list):
            aggregated_str = ""
            for text_file_path in text_file_path_list:
                f = open(text_file_path)
                aggregated_str += f.read()
            return aggregated_str
        
        raw_text = txt_to_str(text_file_path_list)
        if self.add_periods:
            sentences_list = raw_text.split("\n")
            text_list = ["."] + [token 
                for sentence in [sentence.split() + ["."] for sentence in sentences_list] 
                for token in sentence]
        else:
            tokenizer = nltk.tokenize.RegexpTokenizer('[\w\']+|[.$%!?]+')
            if not self.regexp_tokenizer: # Use whitespace tokenizer
                tokenizer = nltk.tokenize.regexp.WhitespaceTokenizer()
            #Include words/numbers as one token, periods/$/% as another.
            #[\w\'] allows us to ensure that contractions like I'm are preserved as one token.
            text_list = tokenizer.tokenize(raw_text)
            if not self.case_sensitive:
                text_list = ["."] + [word.lower() for word in text_list] # lowercase everything
        return text_list

    def list_phrases(self, n):
        """
        Creates a list of tuples of adjacent word pairs.
        
        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> c.text_list = ["This", "is", "Sam", "."]
        >>> c.list_phrases(2)
        [("This", "is"), ("is", "Sam"), ("Sam", ".")]
        """
        return [tuple([self.text_list[i + j] for j in range(n)]) 
            for i in range(0, len(self.text_list) - n + 1)]

    def normalize_dict(self, d):
        """
        Creates copy of d;
        Sums all keys of the dictionary d and divides each key by the total;
        returns the new dict.

        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> d = {1 : 2, 2 : 3}
        >>> c.normalize_dict(d)
        {1 : 0.4, 2 : 0.6}
        """ 
        normalized_dict = dict(d)
        dict_vals_total = sum(d.values())
        normalized_dict.update((key, val / dict_vals_total) for key, val in normalized_dict.items())
        return normalized_dict

    def sample_dict(self, d):
        """
        Sample a key from a dictionary of counts.
        """
        assert d, "dictionary is empty"
        d = self.normalize_dict(d)
        # print(d)
        rand = random.random()

        cumProb = 0.0
        for currKey in d:
            cumProb += d[currKey]
            if rand <= cumProb:
                return currKey
        return currKey

    def count_words(self):
        """
        Returns a dictionary of the words as keys and their counts as values.

        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> c.text_list = ["i", "need", "i", "have"]
        >>> c.count_words()
        {"i": 2, "need": 1, "have": 1}
        """
        ### START YOUR CODE HERE ###
        raise NotImplementedError
        ### END YOUR CODE HERE ###

    def calc_word_probs(self):
        """
        Returns a dictionary with the sample probability of drawing each word.

        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> c.text_list = ["i", "need", "i", "have"]
        >>> c.calc_word_probs()
        {"i": 0.5, "need": 0.25, "have": 0.25}
        """
        ### START YOUR CODE HERE ###
        raise NotImplementedError
        ### END YOUR CODE HERE ###

    def probs_to_neg_log_probs(self, probs_dict):
        """
        Convert dictionary of probabilities into dictionary of negative ln probabilities.
        """
        ### START YOUR CODE HERE ###
        raise NotImplementedError
        ### END YOUR CODE HERE ###
        return neg_log_probs

    def calc_neg_log_word_probs(self):
        """
        Convert text list into dictionary of negative ln probabilities.

        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> c.text_list = ["i", "need", "i", "have"]
        >>> c.calc_neg_log_word_probs()
        {"i": -0.69, "need": -1.39, "have": -1.39}
        """
        word_probs = calc_word_probs()
        return self.probs_to_neg_log_probs(word_probs)

    def calc_adj_counter(self, n):
        """
        Convert text list into dictionary of counts of phrases of length n.

        >>> c = N_gram([], 2)
        >>> c.text_list = ["i", "have", "a", "dream", ".", "i", "have"]
        >>> c.calc_adj_counter(3)
        {("i", "have"): 2, ("have", "a"): 1, ("a", "dream"): 1, ("dream", "."): 1, (".", "i"): 1}
        """
        return collections.Counter(self.list_phrases(n))

    def calc_adj_counters(self, n):
        """
        Create a list of adj_counters of phrase length 1,...,n on text_list.

        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> c.text_list = ["eye", "for", "eye"]
        >>> c.calc_adj_counters(3)
        [0,
        {("eye",): 2, ("for"): 1},
        {("eye", "for"): 1, ("for", "eye"): 1}
        {("eye", "for", "eye"): 1}]
        """
        adj_counters = [0 for i in range(n + 1)]
        for i in range(1, n + 1):
            adj_counters[i] = self.calc_adj_counter(i)
        return adj_counters

    def calc_adj_probs(self, adj_counter):
        """
        Convert adj_counter values from counts to sample probabilities.

        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> adj_counter = {("hey", "you"): 2. ("you", "."): 3}
        >>> c.calc_adj_probs(adj_counter)
        {("hey", "you"): 0.4. ("you", "."): 0.6}
        """
        return self.normalize_dict(adj_counter)

    def filter_adj_counter(self, adj_counter, word_tuple, n):
        """
        Returns: Dictionary with num_occurrences of word pairs starting with 'word_tuple'.

        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> word_tuple = ("Jimmy",)
        >>> adj_counter = {("never", "lie"): 3, ("hey", "Jimmy"): 2, 
            ("Jimmy", "Li"): 1, ("Jimmy", "Carter"): 100}
        >>> c.filter_adj_counter(adj_counter, word_tuple, 2)
        {("Jimmy", "Li"): 1, ("Jimmy", "Carter"): 100}
        """
        assert len(word_tuple) in range(n + 1), \
            "word_tuple does not have valid length: " + str(len(word_tuple))


        if not adj_counter:
            adj_counter = self.calc_adj_counter(n)
        subset_word_adj_counter = {}
        
        if not word_tuple:
            return dict(self.calc_adj_counter(1))
        elif len(word_tuple) == n:
            return {word_tuple: adj_counter[word_tuple]}

        for phrase in adj_counter.keys():
            ### START YOUR CODE HERE ###
            raise NotImplementedError
            ### END YOUR CODE HERE ###
        return subset_word_adj_counter

    def perplexity(self, sentence_list, n):
        """
        Returns perplexity of sentence_list given:
        --text_list: list of tokens to train on.
        --adj_counters: list of all 1,...,n adj_counter. Uses the output of calc_adj_counters(.,.)
        --n: The N in N-gram.

        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> c.train()
        >>> sentence_list = ["super", "confusing", "sentence"]
        >>> c.perplexity(sentence_list, 2)
        2000.01
        """
        text_neg_log_prob = self.calc_neg_log_prob_of_sentence(sentence_list, n, self.p_KN)
        return np.exp(text_neg_log_prob / len(sentence_list))

    def p_naive(self, curr_word, prev_phrase, n):
        """
        Calculates the sample probability of all length-`n` phrases in `text_list`

        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> c.text_list = [".", "i", "think", ".", "i", "am", ".", "why", "yes", "."]
        >>> c.p_naive("i", (".",), 2)
        0.666 # Because 3 length-2 phrases start with (".",), and of the three, 2 of them end with "curr_word"
        """
        assert n >= 1, "Invalid value of n."
        assert len(prev_phrase) < n, "Length of prev_phrase not < n."
        assert isinstance(prev_phrase, tuple), "prev_phrase is not a tuple: " + str(prev_phrase)
        filtered_adj_counter = self.filter_adj_counter(None, prev_phrase, n)
        try:
            ### START YOUR CODE HERE ###
            raise NotImplementedError
            ### END YOUR CODE HERE ###
        except KeyError:
            print(prev_phrase + (curr_word,), " has probability 0.")
            prob = 0
        return prob

    def p_laplace(self, curr_word, prev_phrase, n, k=0.1):
        """
        Calculates the sample probability of all length-`n` phrases in `text_list`
        And performs k-smoothing.
        """
        assert n >= 1, "Invalid value of n."
        assert len(prev_phrase) < n, "Length of prev_phrase not < n."
        assert isinstance(prev_phrase, tuple), "prev_phrase is not a tuple: " + str(prev_phrase)
        
        filtered_adj_counter = self.filter_adj_counter(None, prev_phrase, n)
        num_entries = len(filtered_adj_counter.keys())
        laplace_denominator = sum(filtered_adj_counter.values()) + k * (num_entries + 1)
        smoothed_prob_dict = dict(filtered_adj_counter)
        smoothed_prob_dict.update((key, (val + k) / laplace_denominator) for key, val in smoothed_prob_dict.items())

        unk_prob = k / laplace_denominator # out of distribution words

        err_tolerance = 1e-4
        err_msg = "Smoothed probabilities don't add up to 1!"

        assert np.abs(sum(smoothed_prob_dict.values()) + unk_prob - 1) < err_tolerance, err_msg

        try:
            prob = smoothed_prob_dict[prev_phrase + (curr_word,)]
        except KeyError:
            prob = unk_prob
            # print(prev_phrase + (curr_word,), " has probability 0. Replaced with prob", prob)
        # print("smoothed_prob_dict", smoothed_prob_dict)
        # print("prev_phrase", prev_phrase)
        # print("curr_word", curr_word)
        return prob

    def calc_neg_log_prob_of_sentence(self, sentence_list, n, p_func=p_laplace):
        """
        Return negative log probability of `sentence_list` occurring, given:
        --n: The n in n-gram.
        --p_func: must contain args (curr_word, prev_phrase, text_list, adj_counters, n) in order.
        """
        assert len(sentence_list) > 0, "Empty sentence."
        adj_probs = self.calc_adj_probs(self.adj_counters[n])
        assert len(list(adj_probs.keys())[0]) == n, (
            "Non-matching dimension of adj_probs keys and n.")
        cum_neg_log_prob = 0
        if sentence_list[0] != ".":
            sentence_tuple = (".",) + tuple(sentence_list)
        else:
            sentence_tuple = tuple(sentence_list)

        for i in range(0, len(sentence_tuple) - 1):
            prev_phrase = sentence_tuple[max(0, i - n + 2): i + 1]
            assert len(prev_phrase) < n, ("Invalid length of prev_phrase:" 
                + str(len(prev_phrase)) + ", n: " + str(n))
            curr_word = sentence_tuple[i + 1]

            curr_word_prob = p_func(self, 
                curr_word, 
                prev_phrase, 
                min(len(prev_phrase) + 1, n)
            ) #defaults to using p_naive

            ### START YOUR CODE HERE ###
            raise NotImplementedError
            ### END YOUR CODE HERE ###
        return cum_neg_log_prob

    def calc_prob_of_sentence(self, sentence_list, n, p_func=p_laplace):
        """
        Convert a sentence's neg_log_prob to prob.
        """
        ### START YOUR CODE HERE ###
        raise NotImplementedError
        ### END YOUR CODE HERE ###

    def top_k_adj_starting_with(self, adj_counter, phrase, n, k):
        dict_counter = collections.Counter(self.filter_adj_counter(adj_counter, phrase, n))
        return dict(dict_counter.most_common(k))

    def likeliest_adj_starting_with(self, adj_counter, phrase, n):
        """
        Returns: Tuple with highest num_occurrences starting with `phrase`.
        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> filtered_adj_counter = {("Jimmy", "Li", "is"): 1, ("Jimmy", "Li", "likes"): 100}
        >>> phrase = ("Jimmy", "Li")
        >>> c.likeliest_adj_starting_with(filtered_adj_counter, phrase, 3)
        ("Jimmy", "Li", "likes") # most likely length-3 phrase starting with ("Jimmy", "Li")
        """ 
        assert isinstance(phrase, tuple), "Phrase is not a tuple."
        subset_word_adj_counter = self.filter_adj_counter(adj_counter, phrase, n)
        return max(subset_word_adj_counter, key=lambda adj: subset_word_adj_counter[adj])

    def generate_sentence(self, k=5, T=1):
        """
        Generate a sentence by sampling amongst the top k candidates when generating each token
        """
        assert k >= 1, "k is not at least 1"

        sentence = ""
        prev_phrase = (".",)
        curr_word = "."
        num_words_in_prev_phrase = 1
        while sentence == "" or curr_word != ".":
            n_iter = min(num_words_in_prev_phrase + 1, self.N)

            top_k_adj_counts = self.top_k_adj_starting_with(self.adj_counters[n_iter], prev_phrase, n_iter, k)
            # print("top_k_adj_counts")

            sampled_adj = self.sample_dict(top_k_adj_counts)
            
            prev_phrase, curr_word = sampled_adj[:-1], sampled_adj[-1]
            # sentence += (" " if curr_word != "." and sentence else "") + curr_word
            sentence += " " + curr_word
            if num_words_in_prev_phrase < self.N - 1:
                prev_phrase = prev_phrase + (curr_word,)
            else:
                prev_phrase = prev_phrase[1:] + (curr_word,)

            num_words_in_prev_phrase = min(num_words_in_prev_phrase + 1, self.N)

        return sentence

    def generate_likeliest_sentence(self):
        """
        Naive way of generating likeliest sentence. 
        First chooses the likeliest word following a period (via `p_naive` technique)
        Then chooses the likeliest word following that word.
        Rinse and Repeat.
        Stop when likeliest word is ".".

        >>> c = N_gram([**comma-separated corpora filepaths**], 2)
        >>> c.generate_likeliest_sentence()
        "we have been a year of the people ."
        """
        return self.generate_sentence(1)

In [0]:
class CaseSensitive_N_gram(N_gram):
    def __init__(self, corpora_path_list, N, regexp_tokenizer=True, add_periods=False, case_sensitive=True):
        super().__init__(corpora_path_list, N, regexp_tokenizer, add_periods, case_sensitive)
        self.capsVariations = {}

    def recordCapsVariations(self):
        """
        Record all the capitalization variations of a word in self.capsVariations
        """
        capsVariations = {}

        for token in self.text_list:
            lowercase_token = token.lower()
            if lowercase_token not in capsVariations:
                capsVariations[lowercase_token] = [token] # create a list
            else:
                if token not in capsVariations[lowercase_token]:
                    capsVariations[lowercase_token].append(token)

        return capsVariations

    def caps_filtered_adj_counter(self, adj_counter, word_tuple, n):
        assert len(word_tuple) == n, "len(word_tuple): {} != n: {}".format(len(word_tuple), n)
        prec_phrase, last_word = word_tuple[:-1], word_tuple[-1]
        
        last_word_lowercase = last_word.lower()

        case_adj_counter = {}

        print("self.capsVariations[last_word_lowercase]", self.capsVariations[last_word_lowercase])
        
        for last_word_caps_variation in self.capsVariations[last_word_lowercase]:
            word_tuple_variation = prec_phrase + (last_word_caps_variation,)
            case_adj_counter[word_tuple_variation] = adj_counter[word_tuple_variation]
        return case_adj_counter

    def enumerate_caps_variation_nl_probs(self, adj_counter, sentence_list, n):
        lowercase_sentence_list = [token.lower() for token in sentence_list]
        word_list_caps_variations = []

        for token in lowercase_sentence_list:
            word_list_caps_variations.append(self.capsVariations[token])

        sentence_caps_variations = list(itertools.product(*word_list_caps_variations))

        # print ("sentence_caps_variations", sentence_caps_variations)
        sentence_caps_variations_nl_probs = {}

        for i, sentence_caps_variation in enumerate(sentence_caps_variations):
            if i % 10 == 0:
                print("[{}/{}]".format(i, len(sentence_caps_variations)))
            variation_nl_prob = self.calc_neg_log_prob_of_sentence(list(sentence_caps_variation), n)
            sentence_caps_variations_nl_probs[sentence_caps_variation] = variation_nl_prob

        return sentence_caps_variations_nl_probs

    def truecase_sentence(self, adj_counter, sentence_list, n):
        sentence_caps_variations_nl_probs = self.enumerate_caps_variation_nl_probs(adj_counter, sentence_list, n)
        # print(collections.Counter(sentence_caps_variations_nl_probs).most_common()[-5:])
        return min(sentence_caps_variations_nl_probs, key=lambda sent: sentence_caps_variations_nl_probs[sent])


# Download the Corpus

1. Go [here](https://drive.google.com/open?id=1thAVSW5hzPvNMvvm9RBSkHTHSfTzunfa) to download cs188whw4public.zip. 
2. Extract the zip locally on your computer.
3. Upload the unzipped folder to a folder called "cs188whw4public" in your home directory in google drive. (Please double check you have this step correct! If you get file errors, it's likely because your folder name or folder structure is not correct.)

Then run the following cell and select the Google account that has the folder with files you just created. 



In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path = '/content/drive/My Drive/cs188whw4public/'

# Unit Tests

Please run the primitive unit tests below on your code as a sanity check that you implemented everything correctly.

In [0]:
def test_count_words():
    try:
        c = N_gram([], 2)
        c.text_list = ["i", "need", "i", "have"]
        actual = c.count_words()
        expected = {"i": 2, "need": 1, "have": 1}
        return actual == expected
    except:
        return 0

def test_calc_word_probs():
    try:
        c = N_gram([], 2)
        c.text_list = ["i", "need", "i", "have"]
        actual = c.calc_word_probs()
        expected = {"i": 0.5, "need": 0.25, "have": 0.25}
        return actual == expected
    except:
        return 0

def test_probs_to_neg_log_probs():
    try:
        c = N_gram([], 2)
        c.text_list = ["i", "need", "i", "have"]
        actual = c.probs_to_neg_log_probs(c.calc_word_probs())
        expected = {"i": -np.log(0.5), "need": -np.log(0.25), "have": -np.log(0.25)}
        return actual == expected
    except:
        return 0

def test_filter_adj_counter():
    try:
        c = N_gram([], 2)
        word_tuple = ("Jimmy",)
        adj_counter = {("never", "lie"): 3, ("hey", "Jimmy"): 2, 
            ("Jimmy", "Li"): 1, ("Jimmy", "Carter"): 100}
        actual = c.filter_adj_counter(adj_counter, word_tuple, 2)
        expected = {("Jimmy", "Li"): 1, ("Jimmy", "Carter"): 100}
        return actual == expected
    except:
        return 0

def test_p_naive():
    try:
        c = N_gram([], 2)
        c.text_list = [".", "i", "think", ".", "i", "am", ".", "why", "yes", "."]
        actual = c.p_naive("i", (".",), 2)
        expected = 2/3
        return actual == expected
    except:
        return 0

def test_calc_neg_log_prob_of_sentence():
    try:
        N = 3
        c = N_gram([], N)
        c.text_list = [".", "i", "think", ".", "i", "am", ".", "why", "yes", "."]
        c.train()
        actual = c.calc_neg_log_prob_of_sentence(["i", "am", "yes"], N)
        # assumes laplace smoothing, which is already implemented for you.
        k = 0.1
        expected = sum([-np.log(word_prob) for word_prob in 
                        [(2 + k) / (3 + 3 * k), (1 + k) / (2 + 3 * k), (k / (1 + 2 * k))]
        ])
        return np.abs(actual - expected) < 1e-6
    except:
        return 0

def test_calc_prob_of_sentence():
    try:
        N = 3
        c = N_gram([], N)
        c.text_list = [".", "i", "think", ".", "i", "am", ".", "why", "yes", "."]
        c.train()
        actual = c.calc_prob_of_sentence(["i", "am", "yes"], N)
        # assumes laplace smoothing, which is already implemented for you.
        k = 0.1
        expected = np.prod([(2 + k) / (3 + 3 * k), (1 + k) / (2 + 3 * k), (k / (1 + 2 * k))])
        return np.abs(actual - expected) < 1e-6
    except:
        return 0


In [0]:
# Run the unit tests
q1 = test_count_words()
q2 = test_calc_word_probs()
q3 = test_probs_to_neg_log_probs()
q4 = test_filter_adj_counter()
q5 = test_p_naive()
q6 = test_calc_neg_log_prob_of_sentence()
q7 = test_calc_prob_of_sentence()

func_names = [
              "count_words", 
              "calc_word_probs", 
              "probs_to_neg_log_probs", 
              "filter_adj_counter", 
              "p_naive", 
              "calc_neg_log_prob_of_sentence",
              "calc_prob_of_sentence"
]
results = [q1, q2, q3, q4, q5, q6, q7]

failed_tests = [func_names[i] for i, result in enumerate(results) if not result]


print("========PRIMITIVE UNIT TEST RESULTS=========")
if len(failed_tests) == 0:
    print("YOU GOOD. You passed all the unit tests. You are now ready to generate text!")
else:
    print("YOU NOT GOOD YET. You have some bugs in functions:", failed_tests)

# Text Generation

Here, we will run code that trains an N-gram model from a given corpus of text files and use the learned conditional probability tables to generate some (mediocre) text.

BEFORE RUNNING THE BELOW:
make sure you have filled in the required functions from above!
1. count_words
2. calc_word_probs
3. probs_to_neg_log_probs
4. filter_adj_counter
5. p_naive
6. calc_neg_log_prob_of_sentence
7. calc_prob_of_sentence

In [0]:
training_corpora_path_list = [
    "carter_1980_debate.txt", "carter_apr_1977_energy.txt", "carter_afghan_invasion.txt",
    "carter_notre_dame.txt", "carter_pres_announce_1974.txt", "carter_1979_sotu.txt", 
    "carter_1978_sotu.txt", "carter_inaugural.txt", "carter_1980_sotu.txt", 
    "carter_concession.txt", "carter_crisis_of_confidence.txt", "carter_dnc_1976.txt",
    "carter_dnc_1980.txt", "carter_feb_1977_energy.txt", "carter_panama_signing.txt",
    "carter_nov_1977_energy.txt", "carter_camp_david.txt", "carter_inflation_oct_1978.txt"
]

### REDEFINE training_corpora_path_list here if you wish to use your own corpus.

### END REDEFINE

training_corpora_path_list = [path + file for file in training_corpora_path_list]

### START PLAY WITH PARAMS IN CODE HERE ###
N = 1
k = 1
### END PLAY WITH PARAMS IN CODE HERE ###

carter_ngram = N_gram(training_corpora_path_list, N)
carter_ngram.train()

print("Generated Sentence:", carter_ngram.generate_sentence(k))

# Probabilistic Capitalization

In this section, we will run probabilistic arguments to find the most likely capitalization of an inputted sequence of words.

The example script below will take very long if you input >8 word sentences. You can speed it up by using the Viterbi Algorithm (not implemented here and not in scope this semester for 188).

If you play around, you'll note that this script is not accurate for some slightly out-of-distribution sentences. This can be ameliorated by having a larger training corpus that is ~1-100 million tokens instead of the 50k token corpus we are currently using.

In [0]:
N = 3

carter_ngram_cs = CaseSensitive_N_gram(training_corpora_path_list, N)
carter_ngram_cs.train()

uncapitalized_sentences = [
                           "we are a united people", 
                           "the united states", 
                           "we met soviet challenges in berlin"
]

for original_sentence in uncapitalized_sentences:
    sentence_list = original_sentence.split(" ")
    print("truecased sentence: ", 
        " ".join(list(carter_ngram_cs.truecase_sentence(carter_ngram_cs.adj_counters[N], sentence_list, N))))
    print("original_sentence:", original_sentence)


# Q2: Neural Network Text Generation

After completing Question 2 on the homework, 

In [0]:
!pip install transformers
import argparse
import logging

import numpy as np
import torch
import easydict

from transformers import (
    TransfoXLLMHeadModel,
    TransfoXLTokenizer,
)

In [0]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s", datefmt="%m/%d/%Y %H:%M:%S", level=logging.INFO,
)
logger = logging.getLogger(__name__)

MAX_LENGTH = int(100)  # Hardcoded max length to avoid infinite loop

MODEL_CLASSES = {
    "transfo-xl": (TransfoXLLMHeadModel, TransfoXLTokenizer),
}


def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


#
# Functions to prepare models' input
#


def prepare_ctrl_input(args, _, tokenizer, prompt_text):
    if args.temperature > 0.7:
        logger.info("CTRL typically works better with lower temperatures (and lower top_k).")

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False)
    if not any(encoded_prompt[0] == x for x in tokenizer.control_codes.values()):
        logger.info("WARNING! You are not starting your generation from a control code so you won't get good results")
    return prompt_text


def prepare_xlm_input(args, model, tokenizer, prompt_text):
    # kwargs = {"language": None, "mask_token_id": None}

    # Set the language
    use_lang_emb = hasattr(model.config, "use_lang_emb") and model.config.use_lang_emb
    if hasattr(model.config, "lang2id") and use_lang_emb:
        available_languages = model.config.lang2id.keys()
        if args.xlm_language in available_languages:
            language = args.xlm_language
        else:
            language = None
            while language not in available_languages:
                language = input("Using XLM. Select language in " + str(list(available_languages)) + " >>> ")

        model.config.lang_id = model.config.lang2id[language]
        # kwargs["language"] = tokenizer.lang2id[language]

    # TODO fix mask_token_id setup when configurations will be synchronized between models and tokenizers
    # XLM masked-language modeling (MLM) models need masked token
    # is_xlm_mlm = "mlm" in args.model_name_or_path
    # if is_xlm_mlm:
    #     kwargs["mask_token_id"] = tokenizer.mask_token_id

    return prompt_text


def prepare_xlnet_input(args, _, tokenizer, prompt_text):
    return prompt_text


def prepare_transfoxl_input(args, _, tokenizer, prompt_text):
    return prompt_text


PREPROCESSING_FUNCTIONS = {
    "transfo-xl": prepare_transfoxl_input,
}


def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length  # No generation bigger than model size
    elif length < 0:
        length = MAX_LENGTH  # avoid infinite loop
    return length


def generate_text(prompt_text=""):
    # args = parser.parse_args(argv[1:])
    args = easydict.EasyDict({
        "model_type": "transfo-xl",
        "model_name_or_path" : "transfo-xl-wt103",
        "prompt" : "",
        "length" : 20,
        "stop_token" : None,
        "temperature" : 1.0,
        "repetition_penalty" : 1.0,
        "k" : 0,
        "p" : 0.9,
        "padding_text" : "",
        "xlm_language" : "",
        "seed" : 42,
        "no_cuda" : False,
        "num_return_sequences" : 1,
        "device" : torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "n_gpu" : torch.cuda.device_count(),
    })

    set_seed(args)

    # Initialize the model and tokenizer
    try:
        args.model_type = args.model_type.lower()
        model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
    except KeyError:
        raise KeyError("the model {} you specified is not supported. You are welcome to add it and open a PR :)")

    tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path)
    model = model_class.from_pretrained(args.model_name_or_path)
    model.to(args.device)

    args.length = adjust_length_to_model(args.length, max_sequence_length=model.config.max_position_embeddings)
    logger.info(args)

    # Different models need different input formatting and/or extra arguments
    requires_preprocessing = args.model_type in PREPROCESSING_FUNCTIONS.keys()
    if requires_preprocessing:
        prepare_input = PREPROCESSING_FUNCTIONS.get(args.model_type)
        preprocessed_prompt_text = prepare_input(args, model, tokenizer, prompt_text)
        encoded_prompt = tokenizer.encode(
            preprocessed_prompt_text, add_special_tokens=False, return_tensors="pt", add_space_before_punct_symbol=True
        )
    else:
        encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
    encoded_prompt = encoded_prompt.to(args.device)

    output_sequences = model.generate(
        input_ids=encoded_prompt,
        max_length=args.length + len(encoded_prompt[0]),
        temperature=args.temperature,
        top_k=args.k,
        top_p=args.p,
        repetition_penalty=args.repetition_penalty,
        do_sample=True,
        num_return_sequences=args.num_return_sequences,
    )

    # Remove the batch dimension when returning multiple sequences
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    generated_sequences = []

    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
        generated_sequence = generated_sequence.tolist()

        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # Remove all text after the stop token
        text = text[: text.find(args.stop_token) if args.stop_token else None]

        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
            prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
        )

        generated_sequences.append(total_sequence)
        print(total_sequence)

    return generated_sequences

Change `prompt_text` to whatever phrase you wish the generated sentence to start with.

This script takes really long, ~5-10 mins. It downloads a 1GB Transformer XL model, so every forward pass takes a long time.

In [0]:
generate_text(prompt_text="I am a robot who cannot")

## Fun Resources 

Yay! You've successfully generated text with a probabilistic N-gram model and a pre-trained Neural Network! 

Here are some fun links for exploring more of NLP/AI.  

*  [Talk To Transformer](https://talktotransformer.com/) - Type stuff and a neural network guesses what comes next
* [Thing Translator](https://thing-translator.appspot.com/) - Take pictures of objects and translate them in different languages (use this with your phone)
* [CaptionBot](https://www.captionbot.ai/) - Caption photos
* [Allen NLP demos](https://demo.allennlp.org/) - They have a lot of different demos you can try out on the left sidebar 
* [Talk to Books](https://books.google.com/talktobooks/) - Browse books using everyday language
* [Tensorflow Playground](http://playground.tensorflow.org) - Play around with a neural network in your browser

Take a look at some NLP courses out there. 

 * Stanford [CS 124](http://web.stanford.edu/class/cs124/) - From Languages to Information

* Berkeley [Info 256](http://people.ischool.berkeley.edu/~dbamman/info256.html) - Applied Natural Language Processing

* University of Washington [CSE 517](https://courses.cs.washington.edu/courses/cse517/19wi/) - Natural Language Processing

